In [3]:
import os
import re
from scipy.spatial import KDTree
import pandas as pd
import numpy as np
from datetime import timedelta
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

# --- Utility functions for lat-lon conversion and feature loading ---
def format_coord(val):
    if val.is_integer():
        return f"{int(val)}.0"
    else:
        return f"{val:.2f}"

def ddmm_to_decimal(ddmm):
    degrees = int(ddmm) // 100
    minutes = ddmm - (degrees * 100)
    return round(degrees + minutes / 60, 4)

def load_feature(lat, lon, base_dir='output_features'):
    filename = f'lat_{format_coord(lat)}_lon_{format_coord(lon)}.csv'

    path = os.path.join(base_dir, filename)
    if not os.path.exists(path):
        return None
    df = pd.read_csv(path, parse_dates=['date'])
    df.set_index('date', inplace=True)
    # Drop the date column, keep all feature columns
    return df  # return entire dataframe with all feature columns except date



def parse_coord(filename):
    """
    Extract lat and lon from filename like 'lat_14.0_lon_71.25.csv'
    """
    m = re.match(r'lat_([-+]?\d*\.?\d+)_lon_([-+]?\d*\.?\d+)\.csv', filename)
    if m:
        lat = float(m.group(1))
        lon = float(m.group(2))
        return lat, lon
    return None

def get_available_coords(feature_dir='output_features'):
    """
    Scan feature_dir and return list of (lat, lon) tuples from filenames
    """
    coords = []
    for f in os.listdir(feature_dir):
        coord = parse_coord(f)
        if coord is not None:
            coords.append(coord)
    return coords

def find_nearest_coord(lat, lon, kdtree, coord_array):
    """
    Find nearest available coordinate from kdtree given (lat, lon)
    """
    dist, idx = kdtree.query([lat, lon])
    return tuple(coord_array[idx])

def build_spatiotemporal_tensor(station_lat, station_lon, dates, grid_size=5, feature_dir='output_features'):
    """
    Build spatiotemporal tensor for given station lat/lon and list of dates.
    Finds nearest available feature files around the station within grid_size.
    
    Params:
    - station_lat, station_lon: float, station coordinates
    - dates: list or pd.DatetimeIndex of dates to extract features for
    - grid_size: int, must be odd, grid width around station (e.g. 5 for 5x5)
    - feature_dir: str, directory containing feature CSV files named lat_x_lon_y.csv
    
    Returns:
    - tensor: np.array of shape (len(dates), grid_size*grid_size, num_features)
    """
    offset = grid_size // 2
    desired_grid = [(station_lat + 0.25 * i, station_lon + 0.25 * j)
                    for i in range(-offset, offset + 1)
                    for j in range(-offset, offset + 1)]

    available_coords = get_available_coords(feature_dir)
    if len(available_coords) == 0:
        raise ValueError(f"No feature files found in {feature_dir}")

    coord_array = np.array(available_coords)
    kdtree = KDTree(coord_array)

    tensor_slices = []
    used_coords = set()

    for lat, lon in desired_grid:
        nearest_coord = find_nearest_coord(lat, lon, kdtree, coord_array)
        if nearest_coord in used_coords:
            # Avoid duplicate grid cells if nearest neighbors overlap
            continue
        used_coords.add(nearest_coord)

        filename = f'lat_{nearest_coord[0]}_lon_{nearest_coord[1]}.csv'
        filepath = os.path.join(feature_dir, filename)

        if not os.path.exists(filepath):
            # Should not happen since we got coords from files
            raise ValueError(f"Feature file missing: {filepath}")

        # Read features for the dates
        df = pd.read_csv(filepath, parse_dates=['date']).set_index('date')

        # Ensure all dates present, else you can decide to handle missing ones (e.g. fillna)
        # Select rows for requested dates
        df_sub = df.loc[df.index.isin(dates)]

        if df_sub.empty:
            raise ValueError(f"No feature data found for dates in {filename}")

        # Drop 'date' column if still present; all other columns are features
        # (We set date as index, so no need)
        features = df_sub.values  # shape (num_dates, num_features)

        tensor_slices.append(features)

    # Now tensor_slices is list of arrays (num_dates x num_features) for each grid cell
    # Stack along second dimension: result shape (num_dates, num_grid_cells, num_features)
    tensor = np.stack(tensor_slices, axis=1)

    return tensor


def load_and_binarize_labels(station_file, label_dir='per_station_data', percentile=0.85):
    label_path = os.path.join(label_dir, station_file)
    labels_df = pd.read_csv(label_path, parse_dates=['Date'])
    labels_df.set_index('Date', inplace=True)

    threshold = labels_df['Rainfall'].quantile(percentile)
    labels_df['rainfall_binary'] = (labels_df['Rainfall'] >= threshold).astype(int)
    return labels_df

# --- PyTorch Dataset ---

class RainfallDataset(Dataset):
    def __init__(self, station_files, label_dir='per_station_data', feature_dir='output_features', 
                 T=7, grid_size=5, percentile=0.85):
        self.station_files = station_files
        self.label_dir = label_dir
        self.feature_dir = feature_dir
        self.T = T
        self.grid_size = grid_size
        self.percentile = percentile

        # Preload labels with binary classification for each station
        self.labels_dict = {}
        for sf in station_files:
            self.labels_dict[sf] = load_and_binarize_labels(sf, label_dir, percentile)

        # Build list of (station_file, target_date) pairs for dataset indexing
        self.samples = []
        for sf in station_files:
            label_dates = self.labels_dict[sf].index
            for dt in label_dates:
                # skip dates that don't have enough history for T days
                if dt - timedelta(days=T-1) < label_dates.min():
                    continue
                self.samples.append((sf, dt))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        station_file, target_date = self.samples[idx]

        # Parse lat lon from filename DDMM.M format
        lat_ddmm = float(station_file.split('_')[1])
        lon_ddmm = float(station_file.split('_')[2].replace('.csv', ''))

        station_lat = ddmm_to_decimal(lat_ddmm)
        station_lon = ddmm_to_decimal(lon_ddmm)

        # Build dates for input features
        input_dates = [target_date - timedelta(days=i) for i in reversed(range(self.T))]

        # Build input tensor (T, C, H, W)
        input_tensor = build_spatiotemporal_tensor(station_lat, station_lon, input_dates,
                                                   grid_size=self.grid_size, feature_dir=self.feature_dir)
        input_tensor = torch.tensor(input_tensor, dtype=torch.float32)

        # Get binary label for target date
        label = self.labels_dict[station_file].loc[target_date, 'rainfall_binary']
        label = torch.tensor(label, dtype=torch.float32)

        return input_tensor, label

# --- ConvLSTM model classes (same as before) ---

class ConvLSTMCell(nn.Module):
    def __init__(self, input_channels, hidden_channels, kernel_size):
        super().__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(input_channels + hidden_channels, 4 * hidden_channels, kernel_size, padding=padding)

        self.hidden_channels = hidden_channels

    def forward(self, x, h, c):
        combined = torch.cat([x, h], dim=1)  # concat on channels
        conv_out = self.conv(combined)
        (cc_i, cc_f, cc_o, cc_g) = torch.split(conv_out, self.hidden_channels, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c + i * g
        h_next = o * torch.tanh(c_next)
        return h_next, c_next

class ConvLSTM(nn.Module):
    def __init__(self, input_channels=1, hidden_channels=16, kernel_size=3, num_layers=1, num_classes=1):
        super().__init__()
        self.hidden_channels = hidden_channels
        self.num_layers = num_layers

        self.cell = ConvLSTMCell(input_channels, hidden_channels, kernel_size)
        self.pool = nn.AdaptiveAvgPool2d(1)  # Pool spatial dims to 1x1
        self.fc = nn.Linear(hidden_channels, num_classes)

    def forward(self, x):
        # x shape: (batch, time, channels, height, width)
        batch_size, seq_len, C, H, W = x.size()
        h = torch.zeros(batch_size, self.hidden_channels, H, W, device=x.device)
        c = torch.zeros(batch_size, self.hidden_channels, H, W, device=x.device)

        for t in range(seq_len):
            h, c = self.cell(x[:, t], h, c)

        out = self.pool(h).view(batch_size, -1)
        out = self.fc(out)
        out = torch.sigmoid(out).squeeze(1)  # sigmoid for binary
        return out

# --- Training loop ---

def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * X.size(0)
        preds = (outputs > 0.5).float()
        total_correct += (preds == y).sum().item()
        total_samples += X.size(0)

    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples
    return avg_loss, accuracy

# --- Putting it all together ---

if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # List all station label files you want to train on
    station_files = ['DHARMABAD_1853.0_7750.0.csv']
  # or list manually

    # Create Dataset and DataLoader
    dataset = RainfallDataset(station_files)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

    # Create model, optimizer, loss
    model = ConvLSTM().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.BCELoss()

    epochs = 10
    for epoch in range(epochs):
        loss, acc = train(model, dataloader, optimizer, criterion, device)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {loss:.4f} - Accuracy: {acc:.4f}")


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/manoj/Desktop/MH/.venv/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/manoj/Desktop/MH/.venv/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/tmp/ipykernel_70315/1000158021.py", line 187, in __getitem__
    input_tensor = torch.tensor(input_tensor, dtype=torch.float32)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.
